In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
import pandas as pd
import torch
import numpy as np
import random
import re
import spacy
import shutil
import matplotlib.pyplot as plt
!pip install transformers
!pip install optuna
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertModel
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler
import time
import optuna
from scipy.stats import spearmanr
import psycopg2
import csv
import math
import os
from sqlalchemy import create_engine
from optuna.storages import RDBStorage
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.5 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
path_base = '/content/drive/MyDrive/Tesis/STS_Benchmark/transformer_tunned_BERT/uncase_base/study_optuna/'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!wget -O Stsbenchmark.tar.gz http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
shutil.unpack_archive('./Stsbenchmark.tar.gz', extract_dir='./', format='gztar')

--2023-05-12 01:53:16--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2023-05-12 01:53:17--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   549KB/s    in 0.7s    

2023-05-12 01:53:18 (549 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]



In [5]:
def getSTSBenchmarkSents(filename='sts-train.csv', root='stsbenchmark/', encoding='utf-8'):
  f = open(root+filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for line in f:
    example = re.split(r'\t+', line)
    if len(example) > 7:
      example = example[:-2]
    s2.append(example[-1])
    s1.append(example[-2])
    target.append(float(example[-3]))
  print("{} samples: {}".format(filename, len(target)))
  return s1, s2, target

In [6]:
s1_train,s2_train,target_train = getSTSBenchmarkSents(filename='sts-train.csv')

sts-train.csv samples: 5749


In [7]:
s1_test,s2_test,target_test= getSTSBenchmarkSents(filename='sts-test.csv')

sts-test.csv samples: 1379


In [8]:
s1_dev,s2_dev,target_dev= getSTSBenchmarkSents(filename='sts-dev.csv')

sts-dev.csv samples: 1500


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [10]:
BATCH_SIZE = 32
MAX_LEN = 128
CORPUS = 'STS-B'
BERT_PATH = "bert-base-uncased"

In [11]:
TOKENIZER = BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

In [12]:
def encode_sents(sents1, sents2):
    input_ids_ = []
    attention_masks_ = []
    type_ids_ = []
    for i, sent1 in enumerate(sents1):
        encoded_dict = TOKENIZER.encode_plus(
                            sent1,                      # Sentence 1 to encode.
                            sents2[i],                  # Sentence 2 to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            truncation = True,
                            max_length = MAX_LEN,       # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )
        
        # Add the encoded sentence to the list.    
        input_ids_.append(encoded_dict['input_ids'])
            
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])

        type_ids_.append(encoded_dict['token_type_ids'])
    return input_ids_, attention_masks_, type_ids_

In [13]:
def getEncodedTensors(s1, s2, labels):
    input_ids, attention_masks, type_ids = encode_sents(s1, s2)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    type_ids = torch.cat(type_ids, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, type_ids, labels

In [14]:
input_ids_train,attention_masks_train,type_ids_train,labels_train = getEncodedTensors(s1_train,s2_train,target_train)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval = getEncodedTensors(s1_dev, s2_dev, target_dev)

In [16]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval)

In [17]:
# Create the DataLoaders for our training and test sets.
# We'll take training samples in random order. 
torch.manual_seed(42)
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
            #num_workers=2
        )
torch.manual_seed(42)
val_loader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
            #num_workers=2
        )

In [18]:
# See first batch
#batch = next(iter(train_loader))
#print(batch[0]) # 0 -> input_ids , 1 -> attention_masks, 2 -> type_ids, 3 -> targets
#print(train_loader.batch_size) #tamaño del batch
print('Number of train batches: {}'.format(len(train_loader)))# número de batches
print('Number of evaluation batches: {}'.format(len(val_loader)))

Number of train batches: 180
Number of evaluation batches: 47


In [19]:
class BertBaseUncasedBiLSTMRegressor(nn.Module):
  def __init__(self, output_dim=1, attentions=True, hidden_dim=768, dropout_output=0.1, dropout_lstm=0.1, n_layers = 1, bidirectional=True, freeze=True, batch_first=True):
      super().__init__()
      self.bert = BertModel.from_pretrained(BERT_PATH, output_attentions=attentions) #load the model
      for param in self.bert.parameters():
          param.requires_grad = not freeze #True -> variable, False -> fixed
      self.bilstm = nn.LSTM(input_size=self.bert.config.hidden_size, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout_lstm, batch_first=batch_first)
      self.regressor = nn.Linear(hidden_dim * 2, output_dim)
      self.dropout = nn.Dropout(dropout_output)


  def forward(self, input_ids, type_ids, mask):

      bert_output = self.bert(input_ids=input_ids, token_type_ids = type_ids, attention_mask= mask)
      attention_mask = mask.unsqueeze(-1).expand(bert_output.last_hidden_state.shape).float()
      masked_last_hidden_state = bert_output.last_hidden_state * attention_mask
      """
      The bilstm_output is the tensor of all the hidden state from each time step in the RNN(LSTM), and the hidden state (h_n) returned 
      by the RNN(LSTM) is the last hidden state from the last time step from the input sequence
      """
      bilstm_output, (h_n, _) = self.bilstm(masked_last_hidden_state)
      #h_n = [num layers * num directions, batch size, hid dim]
      # obtain the final hidden states from both directions of the BiLSTM
      final_hidden_state = self.dropout(torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1))

      logits = self.regressor(final_hidden_state)
      return logits, bert_output

In [20]:
def pearson_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)
    return np.corrcoef(all_preds, all_targets)[0, 1]

In [21]:
def spearman_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)

    corr, _ = spearmanr(all_preds, all_targets)
    return corr

In [22]:
def train_loop(model,loader,optimizer,criterion,device,clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    all_preds = []
    all_targets = []
    seed = 42
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    for i, batch in enumerate(loader):
        
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        labels = batch[3].to(device)

        optimizer.zero_grad()
        #Forward 
        outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
        outputs = outputs.squeeze(-1)
        #Loss
        loss = criterion(outputs.view(-1), labels.float())
        #Backprop
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()

        loss_sum += loss.item()

        all_preds.append(outputs.detach().cpu().numpy())
        all_targets.append(labels.float().detach().cpu().numpy())

    epoch_train_loss = loss_sum / len(loader)
    epoch_train_pearson = pearson_corr(all_preds, all_targets)
    epoch_train_spearman = spearman_corr(all_preds, all_targets)

    return epoch_train_loss, epoch_train_pearson, epoch_train_spearman


In [23]:
def evaluation_loop(model,loader,criterion,device):
    #Evaluation loop
    seed = 42
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        all_preds = []
        all_targets = []

        for i, batch in enumerate(loader):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_type_ids = batch[2].to(device)
            labels = batch[3].to(device)

            #Forward
            outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
            outputs = outputs.squeeze(-1)
            #Loss
            loss = criterion(outputs.view(-1), labels.float())

            loss_sum += loss.item()

            all_preds.append(outputs.detach().cpu().numpy())
            all_targets.append(labels.float().detach().cpu().numpy())
        
        epoch_dev_loss = loss_sum / len(loader)
        epoch_dev_pearson = pearson_corr(all_preds, all_targets)
        epoch_dev_spearman = spearman_corr(all_preds, all_targets)
        
    return epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman

In [24]:
def save_metrics_dataframe(metrics_dict, study_name, path_base):
  # Abrir archivo en modo escritura y especificar el separador de campos
  path = path_base + 'metrics_' + study_name + ".csv"
  with open(path, "a+", newline="") as f:
      metrics = [metrics_dict]
      # Crear objeto escritor CSV y especificar el separador de campos
      metrics_csv = csv.DictWriter(f, fieldnames=['train_loss','valid_loss','spearman_train','spearman_val','pearson_train','pearson_val'],delimiter=",")
      
      # Verificar si se ha escrito el encabezado del archivo
      if f.tell() == 0:
        metrics_csv.writeheader()
      
      # Escribir cada fila de datos
      for metric in metrics:
          metrics_csv.writerow(metric)

In [25]:
study_name = 'study-bert-bilstm-hidden-base'

In [26]:
def isNan(value):
  return 0 if math.isnan(value) else value

In [27]:
def objective(trial):
  params = {
      "num_layers": trial.suggest_int("num_layer", 1, 3),
      "dropout_output": trial.suggest_uniform("dropout_output", 0, 0.8),
      "dropout_lstm": trial.suggest_uniform("dropout_lstm", 0, 0.8),
      "freeze" : trial.suggest_categorical('freeze', [True, False]),
      "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
      "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)
  }

  model = BertBaseUncasedBiLSTMRegressor(dropout_output=params["dropout_output"],dropout_lstm=params["dropout_lstm"],freeze=params["freeze"],n_layers = params["num_layers"]).to(device)
  criterion = nn.MSELoss()
  optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])
  
  NUM_EPOCHS = 4
  best_valid_loss = float('inf')
  MODEL_FILE_NAME = CORPUS+'_'+BERT_PATH+'_'+str(MAX_LEN)+'_tunned_model.pt'
  history = {"train": {"loss": []}, "test": {"loss": []}}

  torch.cuda.empty_cache()
  seed = 42
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

  train_loss_list, val_loss_list, spearman_train, spearman_val, pearson_train, pearson_val = [], [], [], [], [], []

  for epoch in range(NUM_EPOCHS):
    
    start_time = time.time()

    epoch_train_loss, epoch_train_pearson, epoch_train_spearman = train_loop(model,train_loader,optimizer,criterion,device)
    epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman = evaluation_loop(model,val_loader,criterion,device)

    elapsed_time = time.time() - start_time

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

    #nos quedamos con el modelo que tiene mejor pérdida de validación
    if epoch_dev_loss < best_valid_loss:
      best_valid_loss = epoch_dev_loss
      torch.save(model.state_dict(), MODEL_FILE_NAME)

    train_loss_list.append(isNan(epoch_train_loss))
    val_loss_list.append(isNan(epoch_dev_loss))
    spearman_train.append(isNan(epoch_train_spearman))
    spearman_val.append(isNan(epoch_dev_spearman))
    pearson_train.append(isNan(epoch_train_pearson))
    pearson_val.append(isNan(epoch_dev_pearson))

    history["train"]["loss"].append(epoch_train_loss)
    history["test"]["loss"].append(epoch_dev_loss)

    print('-' * 80)
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Time: {elapsed_time:.4f}s | Train loss: {epoch_train_loss:.4f} | Dev loss: {epoch_dev_loss:.4f}')
    print(f'Train Pearson Coef: {epoch_train_pearson:.4f} | Dev Pearson Coef: {epoch_dev_pearson:.4f}')
    print(f'Train Spearman Coef: {epoch_train_spearman:.4f} | Dev Spearman Coef: {epoch_dev_spearman:.4f}')

  metrics_dict = {
      'train_loss' : train_loss_list,
      'valid_loss' : val_loss_list,
      'spearman_train': spearman_train,
      'spearman_val': spearman_val,
      'pearson_train': pearson_train,
      'pearson_val': pearson_val
  }

  save_metrics_dataframe(metrics_dict, study_name, path_base)

  return best_valid_loss

In [28]:
#obtener ip de colab para dar acceso a la DB
!curl ipecho.net/plain

34.123.92.166

In [29]:
# Define the connection URL.
url = "postgresql://{user}:{password}@{host}:{port}/{database}"
url = url.format(
    user='postgres',
    password="password",
    host='35.184.132.46',
    port='5432',
    database='stsb-base-bilstm-hidden',
)

# Create the engine and the database (if it doesn't exist yet).
engine = create_engine(url)

# Define the storage.
storage = RDBStorage(url)

In [30]:
# Define el estudio de Optuna utilizando el storage
study = optuna.create_study(study_name=study_name, direction="minimize", storage=storage, load_if_exists=True)

[I 2023-05-12 01:54:13,646] Using an existing study with name 'study-bert-bilstm-hidden-base' instead of creating a new one.


In [31]:
study.optimize(objective, n_trials=5)

<ipython-input-27-3b67103ce24d>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "dropout_output": trial.suggest_uniform("dropout_output", 0, 0.8),
<ipython-input-27-3b67103ce24d>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "dropout_lstm": trial.suggest_uniform("dropout_lstm", 0, 0.8),
<ipython-input-27-3b67103ce24d>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 193.3588s | Train loss: 1.1514 | Dev loss: 0.5629
Train Pearson Coef: 0.6930 | Dev Pearson Coef: 0.8688
Train Spearman Coef: 0.6464 | Dev Spearman Coef: 0.8662
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 194.8895s | Train loss: 0.4082 | Dev loss: 0.7649
Train Pearson Coef: 0.9000 | Dev Pearson Coef: 0.8789
Train Spearman Coef: 0.8785 | Dev Spearman Coef: 0.8773
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 194.8492s | Train loss: 0.2168 | Dev loss: 0.5945
Train Pearson Coef: 0.9483 | Dev Pearson Coef: 0.8872
Train Spearman Coef: 0.9388 | Dev Spearman Coef: 0.8857
--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 195.3101s | Train loss: 0.1420 | Dev loss: 0.4751
Train Pearson Coef: 0.9665 | Dev Pearson Coef: 0.8900
Train Sp

[I 2023-05-12 02:07:43,276] Trial 62 finished with value: 0.4750610979947638 and parameters: {'num_layer': 3, 'dropout_output': 0.772674456423211, 'dropout_lstm': 0.5496704491746849, 'freeze': False, 'optimizer': 'RMSprop', 'learning_rate': 3.40053319012108e-05}. Best is trial 23 with value: 0.4515109192183677.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the c

--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 194.9959s | Train loss: 1.0770 | Dev loss: 0.5851
Train Pearson Coef: 0.7164 | Dev Pearson Coef: 0.8818
Train Spearman Coef: 0.6796 | Dev Spearman Coef: 0.8794
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 194.9904s | Train loss: 0.3493 | Dev loss: 0.4630
Train Pearson Coef: 0.9152 | Dev Pearson Coef: 0.8916
Train Spearman Coef: 0.8951 | Dev Spearman Coef: 0.8879
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 195.3255s | Train loss: 0.1966 | Dev loss: 0.6267
Train Pearson Coef: 0.9533 | Dev Pearson Coef: 0.8891
Train Spearman Coef: 0.9438 | Dev Spearman Coef: 0.8878


...

--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 194.8579s | Train loss: 0.1331 | Dev loss: 0.4765
Train Pearson Coef: 0.9686 | Dev Pearson Coef: 0.8893
Train Spearman Coef: 0.9629 | Dev Spearman Coef: 0.8857


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--------------------------------------------------------------------------------
Epoch: 001/4 | Time: 195.4078s | Train loss: 1.0857 | Dev loss: 0.5907
Train Pearson Coef: 0.7139 | Dev Pearson Coef: 0.8816
Train Spearman Coef: 0.6807 | Dev Spearman Coef: 0.8793
--------------------------------------------------------------------------------
Epoch: 002/4 | Time: 195.3670s | Train loss: 0.3349 | Dev loss: 0.4779
Train Pearson Coef: 0.9188 | Dev Pearson Coef: 0.8879
Train Spearman Coef: 0.8992 | Dev Spearman Coef: 0.8829
--------------------------------------------------------------------------------
Epoch: 003/4 | Time: 194.9158s | Train loss: 0.1902 | Dev loss: 0.6210
Train Pearson Coef: 0.9548 | Dev Pearson Coef: 0.8901
Train Spearman Coef: 0.9448 | Dev Spearman Coef: 0.8884


[I 2023-05-12 03:00:17,125] Trial 66 finished with value: 0.45657997372302606 and parameters: {'num_layer': 3, 'dropout_output': 0.740332779816032, 'dropout_lstm': 0.4887045125208541, 'freeze': False, 'optimizer': 'RMSprop', 'learning_rate': 2.79863051274204e-05}. Best is trial 23 with value: 0.4515109192183677.


--------------------------------------------------------------------------------
Epoch: 004/4 | Time: 194.9292s | Train loss: 0.1360 | Dev loss: 0.4566
Train Pearson Coef: 0.9679 | Dev Pearson Coef: 0.8935
Train Spearman Coef: 0.9621 | Dev Spearman Coef: 0.8891


In [32]:
df_optuna = study.trials_dataframe(attrs=("number", "value", "params", "state")).dropna(subset=['value']) #eliminar valores NaN
# Leer el archivo metrics.CSV y crear un DataFrame
path_metrics = path_base + 'metrics_' + study_name + ".csv"
df_metrics = pd.read_csv(path_metrics).dropna(subset=['train_loss'])
print(len(df_optuna))
print(len(df_metrics ))

65
5


In [ ]:
df_old = pd.DataFrame()

In [33]:
df_old = pd.read_csv(path_base + study_name + "_COMPLETE_SORT.csv")
print("Old trials: ", len(df_old))

Old trials:  60


In [34]:
# Seleccionar los valores de df_optuna que no están ya en df_old
if len(df_optuna)-len(df_metrics) == len(df_old):
    df_optuna_filter = df_optuna.tail(len(df_metrics)).reset_index(drop=True)
else:
    # Seleccionar los valores de df_optuna que no están ya en df_old
    mask = ~df_optuna['value'].isin(df_old['value'])
    df_optuna_filter = df_optuna[mask].reset_index(drop=True)
    print(len(df_optuna_filter))

In [35]:
def concat(df_optuna_filter, df_metrics, df_old):
  # Unir dataframe de metrics con dataframe de optuna de forma horizontal y manejar los valores inexistentes
  df_join = pd.concat([df_optuna_filter, df_metrics], axis=1, sort=False)
  # Unir dataframe de metrics con dataframe de optuna de forma vertical y manejar los valores inexistentes
  df_complete_updated = pd.concat([df_old, df_join], axis=0, sort=False).drop_duplicates()
  print("All trials: ", len(df_complete_updated))
  assert len(df_join) + len(df_old) == len(df_complete_updated)
  return df_complete_updated

In [36]:
if len(df_optuna_filter) == len(df_metrics):
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss', 'value'])

if len(df_optuna_filter) > len(df_metrics):
  num_rows_to_add = len(df_optuna_filter) - len(df_metrics)
  new_rows = [[np.nan] * len(df_metrics.columns) for i in range(num_rows_to_add)]
  new_df = pd.DataFrame(new_rows, columns=df_metrics.columns)
  df_metrics = pd.concat([new_df, df_metrics], ignore_index=True)
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss','value'])

All trials:  65


In [ ]:
df_complete_updated

In [37]:
# Ordenar el DataFrame unido por valor
df = df_complete_updated.sort_values(by='value', ascending=True)
# Obtener la prueba con el mejor valor
best_trial = df.iloc[0]
print("Best trial: ")
print(best_trial)

Best trial: 
number                                                                  23
value                                                             0.451511
params_dropout_lstm                                               0.694133
params_dropout_output                                              0.79315
params_freeze                                                        False
params_learning_rate                                              0.000035
params_num_layer                                                         1
params_optimizer                                                   RMSprop
state                                                             COMPLETE
train_loss               [0.9913704483045472, 0.30517273342443835, 0.16...
valid_loss               [0.5531351363405268, 0.46953036366625034, 0.50...
spearman_train           [0.6971613564415126, 0.9067415771703659, 0.951...
spearman_val             [0.8794959157762853, 0.8864921412388056, 0.888...
pearson_trai

In [38]:
#Guardar CSV completo y ordenado
# ruta de acceso a la carpeta de Google Drive montada
os.remove(path_base + study_name + "_COMPLETE_SORT.csv")
path_ = path_base + study_name + "_COMPLETE_SORT.csv"
# guardar el archivo CSV en Google Drive
df.to_csv(path_, index=False)
files.download(path_)
os.remove(path_metrics)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>